In [1]:
%matplotlib inline
from mpl_toolkits.basemap import Basemap
from umap import UMAP
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.random_projection import SparseRandomProjection
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import itertools as it
import multiprocessing as mp

In [2]:
def parallel(fn, params, n_jobs=mp.cpu_count()):
    pool = mp.Pool(n_jobs)
    print('started new process pool with {} processes'.format(n_jobs))
    try:
        res = pool.map(fn, params)
        pool.close()
        pool.join()
    except:
        print('process pool interrupted, shutting down')
        pool.terminate()
        pool.join()
        raise
    return res

mplog = mp.get_logger()

In [3]:
## Utilities ##

from typing import Dict
from typing import Tuple
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import pairwise_distances
from sklearn.utils.graph_shortest_path import graph_shortest_path
from scipy.stats import pearsonr, spearmanr

def group_dict(iterable, keyfn, mapfn):
    """
    Groups the iterable using the given key function and returns a dictionary
    of keys to groups.
    """
    groups = it.groupby(iterable, key=keyfn)
    gdict = dict()
    for k, g in groups:
        gdict[k] = list(map(mapfn, g))
    return gdict

def stormid_dict(X_df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    """
    Partitions the storm forecast dataset into separate groups for each storm and
    returns the result as a dictionary.
    """
    groups = X_df.groupby(['stormid'])
    storm_dict = dict()
    for stormid, df in groups:
        storm_dict[stormid] = df
    return storm_dict

def feature_groups(X_df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Partitions X_df into three groups by columns:
    1) 0-D features
    2) 11x11 z, u, v wind reanalysis data
    3) 11x11 sst, slp, humidity, and vorticity reanalysis data
    """
    feat_cols = X_df.get(['instant_t', 'windspeed', 'latitude', 'longitude','hemisphere','Jday_predictor','initial_max_wind','max_wind_change_12h','dist2land'])
    nature_cols = pd.get_dummies(X_df.nature, prefix='nature', drop_first=False)
    basin_cols = pd.get_dummies(X_df.basin, prefix='basin', drop_first=False)
    X_0D = pd.concat([feat_cols, nature_cols, basin_cols], axis=1, sort=False)
    X_zuv = X_df.get([col for col in X_df.columns if col.startswith('z_') or col.startswith('u_') or col.startswith('v_')])
    X_sshv = X_df.get([col for col in X_df.columns if col.startswith('sst') or col.startswith('slp')
                   or col.startswith('hum') or col.startswith('vo700')])
    return X_0D, X_zuv, X_sshv

def trust_cont_score(X, X_map, k=10, alpha=0.5, impute_strategy='median'):
    """
    Computes the "trustworthiness" and "continuity" [1] of X_map with respect to X.
    This is a port and extension of the implementation provided by Van der Maaten [2].
    
    Parameters:
    X     : the data in its original representation
    X_map : the lower dimensional representation of the data to be evaluated
    k     : parameter that determines the size of the neighborhood for the T&C measure
    alpha : mixing parameter in [0,1] that determines the weight given to trustworthiness vs. continuity; higher values will give more
            weight to trustworthiness, lower values to continuity.
    
    [1] Kaski S, Nikkilä J, Oja M, Venna J, Törönen P, Castrén E. Trustworthiness and metrics in visualizing similarity of gene expression. BMC bioinformatics. 2003 Dec;4(1):48.
    [2] Maaten L. Learning a parametric embedding by preserving local structure. InArtificial Intelligence and Statistics 2009 Apr 15 (pp. 384-391).
    """
    # Impute X values
    X = Imputer(strategy=impute_strategy).fit_transform(X)
    # Compute pairwise distance matrices
    D_h = pairwise_distances(X, X, metric='euclidean')
    D_l = pairwise_distances(X_map, X_map, metric='euclidean')
    # Compute neighborhood indices
    ind_h = np.argsort(D_h, axis=1)
    ind_l = np.argsort(D_l, axis=1)
    # Compute trustworthiness
    N = X.shape[0]
    T = 0
    C = 0
    t_ranks = np.zeros((k, 1))
    c_ranks = np.zeros((k, 1))
    for i in range(N):
        for j in range(k):
            t_ranks[j] = np.where(ind_h[i,:] == ind_l[i, j+1])
            c_ranks[j] = np.where(ind_l[i,:] == ind_h[i, j+1])
        t_ranks -= k
        c_ranks -= k
        T += np.sum(t_ranks[np.where(t_ranks > 0)])
        C += np.sum(c_ranks[np.where(c_ranks > 0)])
    S = (2 / (N * k * (2 * N - 3 * k - 1)))
    T = 1.0 - S*T
    C = 1.0 - S*C
    return alpha*T + (1.0-alpha)*C

def sammon_stress(X, X_m, impute_strategy='median'):
    X = Imputer(strategy=impute_strategy).fit_transform(X)
    Dx = pairwise_distances(X, X, metric='euclidean')
    Dy = pairwise_distances(X_m, X_m, metric='euclidean')
    # Sammon Stress computes sums over indices where i < j
    # We can interpet this as being the upper triangle of each matrix, from the k=1 diagonal
    Dx_ut = np.triu(Dx, k=1)
    Dy_ut = np.triu(Dy, k=1)
    # Compute Sammon Stress, S
    S = (1 / np.sum(Dx_ut))*np.sum(np.square(Dx_ut - Dy_ut) / (Dx_ut + np.ones(Dx.shape)))
    return S
    
    
def residual_variance(X, X_m, n_neighbors=20):
    kng_h = kneighbors_graph(X, n_neighbors=n_neighbors, mode='distance', n_jobs=mp.cpu_count()).toarray()
    D_h = graph_shortest_path(kng_h, method='D', directed=False)
    #D_h = pairwise_distances(X, X, metric='euclidean')
    #D_l = kneighbors_graph(X_m, n_neighbors=50, mode='distance').toarray()
    D_l = pairwise_distances(X_m, X_m, metric='euclidean')
    r,_ = spearmanr(D_h.flatten(), D_l.flatten())
    return 1 - r**2.0

In [4]:
def pca(X_df, n_components=2):
    imputer = Imputer(strategy='median')
    scaler = StandardScaler()
    pca = PCA(n_components=n_components)
    pca_pipeline = Pipeline([('med_imputer', imputer),('scaler', scaler),('pca',pca)])
    X_pc = pca_pipeline.fit_transform(X_df)    
    return X_pc

def rand_projection(X_df, n_components='auto', eps=0.1):
    imputer = Imputer(strategy='median')
    scaler = StandardScaler()
    proj = SparseRandomProjection(n_components=n_components, eps=eps)
    proj_pipeline = Pipeline([('med_imputer', imputer),('scaler', scaler),('proj', proj)])
    X_rp = proj_pipeline.fit_transform(X_df)
    return X_rp, proj.n_components_

def tsne(X_df, n_components=2, n_iter=5000, perplexity=30, learning_rate=100, init='pca'):
    imputer = Imputer(strategy='median')
    scaler = StandardScaler()
    tsne = TSNE(n_components=n_components, perplexity=perplexity, learning_rate=learning_rate, init=init, n_iter=n_iter)
    tsne_pipeline = Pipeline([('med_imputer', imputer),('scaler', scaler),('tsne', tsne)])
    X_tsne = tsne_pipeline.fit_transform(X_df)
    print("t-SNE completed after {} iterations with final KLD: {}".format(tsne.n_iter_, tsne.kl_divergence_))
    return X_tsne

def umap(X_df, n_components=2, y=None, n_neighbors=5, min_dist=0.1, metric='correlation'):
    import warnings
    warnings.filterwarnings('ignore')
    imputer = Imputer(strategy='median')
    scaler = StandardScaler()
    umap = UMAP(n_components=n_components, n_neighbors=n_neighbors, min_dist=min_dist, metric=metric)
    umap_pipeline = Pipeline([('med_imputer', imputer),('scaler', scaler),('umap', umap)])
    X_umap = umap_pipeline.fit_transform(X_df, y)
    warnings.resetwarnings()
    return X_umap

def evaluate(X, X_maps, k=20, top_n=1, alpha=0.5):
    """
    Evaluates a collection of X_maps relative to X using the trustworthiness/continuity metric.
    Returns 'top_n' tuples of (X_map, TC score, index)
    
    X      : the original, untransformed data
    X_maps : an iterable of lower dimensional mappings to evaluate
    k      : neighborhood parameter for T&C metric
    top_n  : # of mappings to return, in descending order of score
    """
    assert top_n <= len(X_maps)
    scores = np.array([trust_cont_score(X, X_m, k=k, alpha=alpha) for X_m in X_maps])
    sorted_inds = np.argsort(scores)[::-1]
    sorted_scores = scores[sorted_inds]
    sorted_maps = [X_maps[i] for i in sorted_inds]
    return list(zip(sorted_maps, sorted_scores, sorted_inds))[:top_n]

def plot_mapping_vs_y_2d(X_map, ys, title="", xlabel="", ylabel="", instant_labels: pd.DataFrame = None):
    plt.scatter(X_map[:,0], X_map[:,1], c=ys, cmap='copper')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.colorbar().set_label("Ice area")
    if instant_labels is not None:
        for (i, xy) in enumerate(X_map):
            if i % 2 == 0:
                plt.annotate(instant_labels.values[i], xy)

In [5]:
X_ds = xr.open_dataset('data/train.nc')
ys = np.load('data/train.npy') 

In [6]:
X_arr = X_ds.to_array()
print(X_arr.shape)

(8, 1500, 39, 58)


### Per Variable analysis

Produce 2D visualizations for each individual variable

In [8]:
def run_rp(params):
    X, d = params
    return rand_projection(X, n_components=d)[0]

def run_tsne(params):
    X, d, perp = params
    # Reduce dims with PCA
    X_pc = pca(X, n_components=50)
    return tsne(X_pc, perplexity=perp, init='random')

def run_umap(params):
    X, d, nn, min_dist = params
    return umap(X, n_components=d, n_neighbors=nn, min_dist=min_dist)

def run_rp_with_tc_cv(X, d=2, n_runs=24, k=20):
    X_rps = parallel(run_rp, [(X, d)]*n_runs)
    X_rp, tc, _ = evaluate(X, X_rps, k=k)[0]
    return X_rp, tc

def run_tsne_with_tc_cv(X, perplexities, d=2, k=20):
    params = it.product([X], [d], perplexities)
    X_tsnes = parallel(run_tsne, params)
    X_tsne, tc, i = evaluate(X, X_tsnes, k=k)[0]
    return X_tsne, tc, params[i][1:]

def run_umap_with_tc_cv(X, n_neighbors, min_dists, d=2, k=20):
    params = it.product([X], [d], n_neighbors, min_dists)
    X_umaps = parallel(run_umap, params)
    X_umap, tc, i = evaluate(X, X_umaps, k=k)[0]
    return X_umap, tc, params[i][1:]

In [ ]:
data_vars = list(X_ds.data_vars.keys())
TC_k = 20
tsne_perplexities = range(20, 100, 20)
umap_nns = [10, 20, 50]
umap_mds = np.arange(0.1, 1.0, 0.2)
res_dict = dict()
for (i, (var_name, X_var_3d)) in enumerate(zip(data_vars, X_arr.values)):
    print('Running single variable tests for {}'.format(var_name))
    # Flatten spatial dimensions
    n_t, n_lat, n_lon = X_var_3d.shape[0], X_var_3d.shape[1], X_var_3d.shape[2]
    X_var = X_var_3d.reshape((n_t, n_lat*n_lon))
    assert(X_var.shape == (n_t, n_lat*n_lon))
    res = []
    # PCA
    X_var_pc = pca(X_var, n_components=2)
    tc_pc = trust_cont_score(X_var, X_var_pc, k=TC_k)
    res.append((X_var_pc, tc_pc, "PCA"))
    # Random projections
    X_var_rp, tc_rp = run_rp_with_tc_cv(X_var, k=TC_k)
    res.append((X_var_rp, tc_rp, "RP"))
    # t-SNE
    X_var_tsne, tc_tsne = run_tsne_with_tc_cv(X_var, tsne_perplexities, k=TC_k)
    res.append((X_var_tsne, tc_tsne, "t-SNE"))
    # UMAP
    X_var_umap, tc_umap = run_umap_with_tc_cv(X_var, umap_nns, umap_mds, k=TC_k)
    res.append((X_var_umap, tc_umap, "UMAP"))
    # Store results for variable
    res_dict[var_name] = res


Running single variable tests for ice_area
started new process pool with 8 processes
started new process pool with 8 processes
t-SNE completed after 2049 iterations with final KLD: 2.539768934249878
t-SNE completed after 1699 iterations with final KLD: 2.2243263721466064
t-SNE completed after 1699 iterations with final KLD: 1.983177900314331
t-SNE completed after 4999 iterations with final KLD: 2.975492000579834
started new process pool with 8 processes
Running single variable tests for ts
started new process pool with 8 processes
started new process pool with 8 processes
t-SNE completed after 4999 iterations with final KLD: 2.1451497077941895
t-SNE completed after 4999 iterations with final KLD: 2.0397424697875977
t-SNE completed after 4999 iterations with final KLD: 1.9198139905929565
t-SNE completed after 4349 iterations with final KLD: 1.7984490394592285
started new process pool with 8 processes
Running single variable tests for taux
started new process pool with 8 processes
starte

In [ ]:
n_plots_per_var = 4
plt.figure(figsize=(25,45))
for (i, (var_name, maps)) in enumerate(res_dict.items()):
    for (j, (X_2d, tc, name)) in enumerate(maps):
        plt.subplot(len(data_vars), n_plots_per_var, i*n_plots_per_var + j + 1)
        plot_mapping_vs_y_2d(X_2d, ys, "{}, {} 2D (TC={:.3f})".format(var_name, name, tc))
        
plt.show()

### Full feature set

In [9]:
# Swap variable and time axes
X_swap = np.swapaxes(X_arr.values, 0, 1)
# Flatten variable and spatial dimensions
n_t, n_vars, n_lat, n_lon = X_swap.shape[0], X_swap.shape[1], X_swap.shape[2], X_swap.shape[3]
X_full = X_swap.reshape((n_t, n_vars*n_lat*n_lon))
assert(X_full.shape == (n_t, n_vars*n_lat*n_lon))

In [8]:
TC_k = 20
tsne_perplexities = range(20, 100, 20)
umap_nns = [10, 20, 50]
umap_mds = np.arange(0.1, 1.0, 0.2)
res = []
# PCA
print('Running PCA on full feature set')
X_full_pc = pca(X_full, n_components=2)
tc_pc = trust_cont_score(X_full, X_full_pc, k=TC_k)
res.append((X_full_pc, tc_pc, "PCA"))
# Random projections
print('Running RP on full feature set')
X_full_rp, tc_rp = run_rp_with_tc_cv(X_full, k=TC_k)
res.append((X_full_rp, tc_rp, "RP"))
# t-SNE
print('Running t-SNE on full feature set')
X_full_tsne, tc_tsne, params_tsne = run_tsne_with_tc_cv(X_full, tsne_perplexities, k=TC_k)
res.append((X_full_tsne, tc_tsne, "t-SNE"))
print(params_tsne)
# UMAP
print('Running UMAP on full feature set')
X_full_umap, tc_umap, params_umap = run_umap_with_tc_cv(X_full, umap_nns, umap_mds, k=TC_k)
res.append((X_full_umap, tc_umap, "UMAP"))
print(params_umap)

Running PCA on full feature set


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(30, 5))
plt.subplot(1, 4, 1)
plot_mapping_vs_y_2d(X_full_pc, ys, "PCA 2D (TC={:.3f})".format(tc_pc))
plt.subplot(1, 4, 2)
plot_mapping_vs_y_2d(X_full_rp, ys, "RP 2D (TC={:.3f})".format(tc_rp))
plt.subplot(1, 4, 3)
plot_mapping_vs_y_2d(X_full_tsne, ys, "t-SNE 2D (TC={:.3f})".format(tc_tsne))
plt.subplot(1, 4, 4)
plot_mapping_vs_y_2d(X_full_umap, ys, "UMAP 2D (TC={:.3f})".format(tc_umap))
plt.show()

### Analysis of performance in higher dimensions

In [18]:
d_hi = X_full.shape[1]
log_lim = np.floor(np.log2(d_hi)) - 1
ds = np.logspace(0, log_lim, num=log_lim + 1, base=2.0)
X_hi = X_full

In [19]:
# PCA
def run_pca(d):
    return pca(X_hi, d)

# Random projection parameters
n_runs = 20
rp_params = ds
def run_rp(d):
    return rand_projection(X_hi, d)

# UMAP parameters
n_neighbors = np.arange(20, 100, 20)
min_dists = np.arange(0.1, 1.0, 0.2)
umap_params = list(it.product(ds, n_neighbors, min_dists))
def run_umap(x):
    d, nn, md = x
    return umap(X_hi, n_components=d, n_neighbors=nn, min_dist=md)

In [ ]:
print('Running PCA with {} configurations'.format(len(ds)))
X_lo_pca = parallel(run_pca, ds)
res_pca = list(zip(ds, X_lo_pca))
print('Running RP with {} configurations'.format(len(ds)))
X_lo_rp = parallel(run_rp, ds)
res_rp = list(zip(ds, X_lo_rp))
#print('Running t-SNE with {} configurations'.format(len(tsne_params)))
#res_tsne = zip(tsne_params, parallel(run_tsne, tsne_params))
print('Running UMAP with {} configurations'.format(len(umap_params)))
X_lo_umap = parallel(run_umap, umap_params)
res_umap = list(zip(ds, X_lo_umap))
print('Done!')

In [ ]:
pca_groups = group_dict(res_pca, lambda x: x[0], lambda x: x[1])
rp_groups = group_dict(res_rp, lambda x: x[0], lambda x: x[1])
umap_groups = group_dict(res_umap, lambda x: x[0][0], lambda x: x[1])

def compute_trustworthiness_scores(d):
    print('Computing trustworthiness values for d={}'.format(d))
    _, tc_pca, _ = evaluate(X_hi, pca_groups[d], k=10)[0]
    _, tc_rp, _ = evaluate(X_hi, rp_groups[d], k=10)[0]
    _, tc_umap, i = evaluate(X_hi, umap_groups[d], k=10)[0]
    print('best params UMAP: {}'.format(umap_params[i]))
    return tc_pca, tc_rp, tc_umap

ts_all = parallel(compute_trustworthiness_scores, ds)
ts_pca, ts_rp, ts_umap = zip(*ts_all)

In [ ]:
plt.plot(ds, ts_pca)
plt.plot(ds, ts_rp)
plt.plot(ds, ts_umap)
plt.legend(['PCA', 'RP', 'UMAP'])
plt.title('T&C score vs d for 0D features\n (higher is better)')
plt.xlabel('Dimensionality of embdding (d)')
plt.ylabel('Trust/Cont score (k=10)')
plt.show()